In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [23]:
raw_data = pd.read_csv('MyScrape.csv')

In [24]:
raw_data['Model'] = raw_data['web-scraper-start-url'].str.extract(r'aspect=Modell%3A%3A%3A([^&]+)')
raw_data['Storage'] = raw_data['web-scraper-start-url'].str.extract(r'aspect=Speicherkapazit%C3%A4t%3A%3A%3A([^&]+)')
raw_data['Model'] = raw_data['Model'].str.replace('+', ' ')
raw_data['Storage'] = raw_data['Storage'].str.replace('+', ' ')
raw_data['Condition'] = 'gebraucht'

In [25]:
conditions = {
    'neu': 'neu',
    'neu sonstige': 'neu sonstige',
    'zertifiziert refurbished': 'zertifiziert refurbished',
    'hervorragend refurbished': 'hervorragend refurbished',
    'sehr gut refurbished': 'sehr gut refurbished',
    'gut refurbished': 'gut refurbished',
    'gebraucht': 'gebraucht',
    'defekt': 'defekt'
}
for condition, keyword in conditions.items():
    raw_data.loc[raw_data['Listing'].str.contains(keyword, case=False), 'Condition'] = condition

In [26]:
raw_data.drop(columns=['web-scraper-order', 'web-scraper-start-url', 'Listing'], inplace=True)

In [27]:

raw_data

,Price,Sale_Format,Shipping_Cost,Shipping_percent,Shipping,Total sold,Item_Revenue,Bids,Date,Model,Storage,Condition
0,912.64,Fixed price,0.0,1.0,Free shipping,1,912.64,–,11/23/2020,Apple iPhone 11 Pro Max,64 GB,gebraucht
1,1231.63,Fixed price,0.0,1.0,Free shipping,1,1231.63,–,11/23/2020,Apple iPhone 11 Pro Max,256 GB,gebraucht
2,1231.63,Fixed price,0.0,1.0,Free shipping,1,1231.63,–,11/23/2020,Apple iPhone 11 Pro Max,512 GB,gebraucht
3,970.84,Fixed price,0.0,1.0,Free shipping,2,1941.68,–,11/23/2020,Apple iPhone 11 Pro,64 GB,gebraucht
4,1095.31,Fixed price,0.0,1.0,Free shipping,4,4381.24,–,11/23/2020,Apple iPhone 11 Pro,512 GB,gebraucht
...,...,...,...,...,...,...,...,...,...,...,...,...
81267,1282.98,Fixed price,0.0,1.0,Free shipping,9,11546.84,–,11/23/2023,Apple iPhone 14 Pro Max,256 GB,neu
81268,977.93,Fixed price,0.0,1.0,Free shipping,1,977.93,–,11/23/2023,Apple iPhone 14 Pro Max,256 GB,gebraucht
81269,977.88,Fixed price,0.0,1.0,Free shipping,1,977.88,–,11/23/2023,Apple iPhone 14 Pro Max,256 GB,gebraucht
81270,1260.76,Fixed price,0.0,1.0,Free shipping,1,1260.76,–,11/23/2023,Apple iPhone 14 Pro Max,256 GB,neu


In [28]:
common_versions = {
    'Apple iPhone 11 Pro Max': '20.09.2019',
    'Apple iPhone 11 Pro': '20.09.2019',
    'Apple iPhone XS': '26.10.2018',
    'Apple iPhone XR': '26.10.2018',
    'Apple iPhone 11': '20.09.2019',
    'Apple iPhone X': '03.11.2017',
    'Apple iPhone 12': '23.10.2020',
    'Apple iPhone 12 Pro': '23.10.2020',
    'Apple iPhone 12 Pro Max': '23.10.2020',
    'Apple iPhone 12 mini': '23.10.2020',
    'Apple iPhone 13 mini': '24.09.2021',
    'Apple iPhone 13 Pro': '24.09.2021',
    'Apple iPhone 13 Pro Max': '24.09.2021',
    'Apple iPhone 13': '24.09.2021',
    'Apple iPhone 14 Pro': '16.09.2022',
    'Apple iPhone 14 Pro Max': '16.09.2022',
    'Apple iPhone 14': '16.09.2022'
}
raw_data['Release_date'] = raw_data['Model'].map(common_versions)

In [29]:
raw_data

,Price,Sale_Format,Shipping_Cost,Shipping_percent,Shipping,Total sold,Item_Revenue,Bids,Date,Model,Storage,Condition,Release_date
0,912.64,Fixed price,0.0,1.0,Free shipping,1,912.64,–,11/23/2020,Apple iPhone 11 Pro Max,64 GB,gebraucht,20.09.2019
1,1231.63,Fixed price,0.0,1.0,Free shipping,1,1231.63,–,11/23/2020,Apple iPhone 11 Pro Max,256 GB,gebraucht,20.09.2019
2,1231.63,Fixed price,0.0,1.0,Free shipping,1,1231.63,–,11/23/2020,Apple iPhone 11 Pro Max,512 GB,gebraucht,20.09.2019
3,970.84,Fixed price,0.0,1.0,Free shipping,2,1941.68,–,11/23/2020,Apple iPhone 11 Pro,64 GB,gebraucht,20.09.2019
4,1095.31,Fixed price,0.0,1.0,Free shipping,4,4381.24,–,11/23/2020,Apple iPhone 11 Pro,512 GB,gebraucht,20.09.2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81267,1282.98,Fixed price,0.0,1.0,Free shipping,9,11546.84,–,11/23/2023,Apple iPhone 14 Pro Max,256 GB,neu,16.09.2022
81268,977.93,Fixed price,0.0,1.0,Free shipping,1,977.93,–,11/23/2023,Apple iPhone 14 Pro Max,256 GB,gebraucht,16.09.2022
81269,977.88,Fixed price,0.0,1.0,Free shipping,1,977.88,–,11/23/2023,Apple iPhone 14 Pro Max,256 GB,gebraucht,16.09.2022
81270,1260.76,Fixed price,0.0,1.0,Free shipping,1,1260.76,–,11/23/2023,Apple iPhone 14 Pro Max,256 GB,neu,16.09.2022


In [30]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Load the data
# Assuming your dataframe is named 'raw_data'
# You might need to preprocess the data further depending on the characteristics of your dataset
# For example, converting categorical variables to numerical, handling missing values, etc.
# For simplicity, let's assume you have already done the necessary preprocessing

# Extract features and target variable
features = ['Sale_Format', 'Shipping_Cost', 'Shipping_percent', 'Total sold', 'Item_Revenue']
target = 'Price'

X = raw_data[features]
y = raw_data[target]

# Convert categorical variables to numerical using one-hot encoding
X = pd.get_dummies(X, columns=['Sale_Format'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.7809195139957072


In [34]:
predictions

array([797.4979, 184.1282, 330.5674, ..., 322.2499, 386.9598, 265.7658])

In [60]:
data = {'Shipping_Cost': [19.75],
        'Shipping_percent': [12.0],
        'Total sold': [2],
        'Item_Revenue': [1212.51],
        'Sale_Format_Auction': [False],
        'Sale_Format_Fixed price': [True]}
predict=pd.DataFrame(data)
print(predict)

   Shipping_Cost  Shipping_percent  Total sold  Item_Revenue  \
0          19.75              12.0           2       1212.51   

   Sale_Format_Auction  Sale_Format_Fixed price  
0                False                     True  


In [61]:
model.predict(predict)

array([606.3081])